In [2]:
vs = '''
#version 330

uniform mat4 projection;
uniform mat4 view;

in vec3 in_position;
in vec3 in_normal;
in vec2 in_texcoord_0;

out vec3 v_vert;
out vec3 v_norm;
out vec2 texcoord_0;

void main() {
    gl_Position = (projection * view) * vec4(in_position, 1.0);
    v_vert = in_position;
    v_norm = in_normal;
    texcoord_0 = in_texcoord_0;
}
'''

fs = '''
#version 330

uniform vec3 Light;

uniform sampler2D Texture;

struct Material {
    sampler2D diffuse;
    vec3      specular;
    float     shininess;
};
uniform Material material;

struct DirLight {
    vec3 direction;
    vec3 ambient;
    vec3 diffuse;
    vec3 specular;
};
uniform DirLight dirLight;

in vec3 v_vert;
in vec3 v_norm;
in vec2 texcoord_0;

out vec4 f_color;


vec3 CalcDirLight(DirLight light, vec3 normal, vec3 viewDir)
{
    vec3 direction = normalize(-light.direction);
    float lum = clamp(dot(direction, normal), 0.0, 1.0);
    float spec = pow(max(dot(viewDir, reflect(direction, normal)), 0.0), material.shininess);

    vec3 difftex = vec3(texture(material.diffuse, texcoord_0));
    //vec3 spectex = vec3(texture(material.specular, texcoord_0));
    vec3 spectex = material.specular;
    
    vec3 ambient = light.ambient * difftex;
    vec3 diffuse = light.diffuse * lum * difftex;
    vec3 specular = light.specular * spec * spectex;    
    return (ambient + diffuse + specular);
}


void main() {
    vec3 viewDir = normalize(vec3(-1, 1,-1));
    f_color = vec4(CalcDirLight(dirLight, normalize(v_norm), viewDir), 1.0);
}
'''

def callback(program):
    program["dirLight.direction"].value = (-2.0,-1.0,-3.0)
    program["dirLight.diffuse"].value   = ( 1.0, 1.0, 1.0)
    program["dirLight.ambient"].value   = ( 0.8, 0.8, 0.8)
    program["dirLight.specular"].value  = ( 0.8, 0.8, 0.8)
    program["material.specular"].value  = ( 0.9, 0.9, 0.9)
    program["material.shininess"].value = 100

vs = '../meshmaker/shaders/main.vs'
fs = '../meshmaker/shaders/main.fs'
    
primary_shader = dict(vertex_shader=vs, fragment_shader=fs, callback=callback,
                      signature=('in_texcoord_0', 'in_normal', 'in_position'))

In [7]:
from meshmaker.base import Base
import moderngl
import os

class ShaderProgram(Base):
    
    def _readiffile(self, s):
        if s is not None:
            return open(s, 'r').read().strip() if os.path.isfile(s) else s
    
    def __init__(self,
                 vs='../meshmaker/shaders/main.vs', gs=None,
                 fs='../meshmaker/shaders/main.fs',
                 signature=('in_texcoord_0', 'in_normal', 'in_position')):
        self.vs = vs
        self.gs = gs
        self.fs = fs
        self.signature = signature
    
    def build(self, ctx):
        """Compute shader program from context and associated GLSL scripts"""
        vs = self._readiffile(self.vs)
        gs = self._readiffile(self.gs)
        fs = self._readiffile(self.fs)
        self._ctx = ctx
        self._program = ctx.program(
            vertex_shader=vs,
            geometry_shader=gs,
            fragment_shader=fs)
        return self
    
    def init(self):
        """Set GLSL uniform variables at start"""
        self._program["dirLight.direction"].value = (-2.0,-1.0,-3.0)
        self._program["dirLight.diffuse"].value   = ( 1.0, 1.0, 1.0)
        self._program["dirLight.ambient"].value   = ( 0.2, 0.2, 0.2)
        self._program["dirLight.specular"].value  = ( 0.8, 0.8, 0.8)
        self._program["material.specular"].value  = ( 0.9, 0.9, 0.9)
        self._program["material.shininess"].value = 100
    
    def update(self, camera):
        """Set GLSL uniform variables per frame for camera"""
        self._program['projection'].write(camera.projection.astype('f4').tobytes())        
        self._program['view'].write(camera.view.astype('f4').tobytes())
        self._program['viewDir'].value = tuple(camera.viewdir)
        
    def render(self, vertexdata, material):
        """Compute function which renders vertexdata using shaders"""
        vertexdata = np.array(vertexdata)
        vbo = self._ctx.buffer(vertexdata.astype('f4').tobytes())
        vao = self._ctx.simple_vertex_array(self._program, vbo, *self.signature)
        
        def f():
            if material:
                material.use()
            vao.render(moderngl.TRIANGLES)
        
        return f
    
sps = [ShaderProgram()]
GLWindow.test('../resources/textures', programs=sps)

Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


In [7]:
vs = '''
#version 330 core
layout (location = 0) in vec3 aPos;
layout (location = 1) in vec3 aNormal;

out VS_OUT {
    vec3 normal;
} vs_out;

uniform mat4 projection;
uniform mat4 view;

void main()
{
    gl_Position = projection * view * vec4(aPos, 1.0);
    mat3 normalMatrix = mat3(transpose(inverse(view)));
    vs_out.normal = normalize(vec3(projection * vec4(normalMatrix * aNormal, 0.0)));
}
'''

gs = '''
#version 330 core
layout (triangles) in;
layout (line_strip, max_vertices = 6) out;

in VS_OUT {
    vec3 normal;
} gs_in[];

const float MAGNITUDE = 0.4;

void GenerateLine(int index)
{
    gl_Position = gl_in[index].gl_Position;
    EmitVertex();
    gl_Position = gl_in[index].gl_Position + vec4(gs_in[index].normal, 0.0) * MAGNITUDE;
    EmitVertex();
    EndPrimitive();
}

void main()
{
    GenerateLine(0); // first vertex normal
    GenerateLine(1); // second vertex normal
    GenerateLine(2); // third vertex normal
}  
'''

fs = '''
#version 330 core
out vec4 FragColor;

void main()
{
    FragColor = vec4(1.0, 0.0, 0.0, 1.0);
}
'''

def callback(program):
    pass

secondary_shader = dict(vertex_shader=vs, geometry_shader=gs,
                        fragment_shader=fs, callback=callback,
                        signature=('aNormal', 'aPos'))

In [8]:
shaders = [primary_shader, secondary_shader]
GLWindow.test('../resources/textures', shaders=shaders)

Error: GLSL Compiler failed

vertex_shader
=============
0:1(1): error: syntax error, unexpected DOT_TOK



In [1]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.tform import TForm
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.geometry import batch, slide, loop_offset, loop_normal, loop_contains, isnear, near, loopO
from meshmaker.mgl import Window as GLWindow, show
from meshmaker.plt import *
from collections import defaultdict
import json

GLWindow.test('../resources/textures')

Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


In [19]:
vs='''
#version 330 core
layout (location = 1) in vec3 in_position;
layout (location = 0) in vec3 in_normal;

out VS_OUT {
    vec3 normal;
} vs_out;

uniform mat4 projection;
uniform mat4 view;
//uniform mat4 model;

void main()
{
    //gl_Position = projection * view * model * vec4(aPos, 1.0); 
    gl_Position = projection * view * vec4(in_position, 1.0); 
    //mat3 normalMatrix = mat3(transpose(inverse(view * model)));
    mat3 normalMatrix = mat3(transpose(inverse(view)));
    vs_out.normal = normalize(vec3(projection * vec4(normalMatrix * in_normal, 0.0)));
}
'''

gs='''
#version 330 core
layout (triangles) in;
layout (line_strip, max_vertices = 6) out;

in VS_OUT {
    vec3 normal;
} gs_in[];

const float MAGNITUDE = 0.4;

void GenerateLine(int index)
{
    gl_Position = gl_in[index].gl_Position;
    EmitVertex();
    gl_Position = gl_in[index].gl_Position + vec4(gs_in[index].normal, 0.0) * MAGNITUDE;
    EmitVertex();
    EndPrimitive();
}

void main()
{
    GenerateLine(0); // first vertex normal
    GenerateLine(1); // second vertex normal
    GenerateLine(2); // third vertex normal
}
'''

fs='''
#version 330 core
out vec4 FragColor;

void main()
{
    FragColor = vec4(1.0, 0.0, 0.0, 1.0);
}
'''

shaders = dict(signature=('in_position', 'in_normal'),
               vertex_shader=vs, geometry_shader=gs, fragment_shader=fs)

In [30]:
def callback(prog):
    prog["dirLight.direction"].value = (-2.0, -1.0, -3.0)
    prog["dirLight.ambient"].value  = (0.5, 0.5, 0.5)
    prog["dirLight.diffuse"].value  = (1.0, 1.0, 1.0)
    prog["dirLight.specular"].value = (0.8, 0.8, 0.8)
    
    #prog["Light.Position"].value = (-2.0, -1.0, -3.0, 0.0)
    #prog["Light.Direction"].value = (-2, -1, -3)
    #prog["Light.Intensity"].value = (1, 1, 1)
    #prog["material.Kd"].value = (0.9, 0.9, 0.9)
    #prog["material.Ks"].value = (0.9, 0.9, 0.9)
    #prog["material.specular"].value = (0.1, 0.1, 0.1)
    #prog["material.shininess"].value = 100


shaders = dict(
vertex_shader=''' 
#version 330

uniform mat4 projection;
uniform mat4 view;

in vec3 in_position;
in vec3 in_normal;
in vec2 in_texcoord_0;

out vec3 position;
out vec3 normal;
out vec2 texcoord_0;

void main()
{
    gl_Position = projection * view * vec4(in_position, 1.0);
    position = vec3(projection * view * vec4(in_position, 1.0));
    //normal = normalize( NormalMatrix * in_normal);
    normal = normalize(in_normal);
    texcoord_0 = in_texcoord_0;
}
''',
fragment_shader='''
#version 330

in vec3 position;
in vec3 normal;
in vec2 texcoord_0;

layout (location = 0) out vec4 FragColor;

//layout(binding=0) uniform sampler2D Tex1;
//uniform sampler2D Tex1;

struct Material {
    sampler2D diffuse;
    vec3      specular;
    float     shininess;
};
uniform Material material;

struct DirLight {
    vec3 direction;
    vec3 ambient;
    vec3 diffuse;
    vec3 specular;
};
uniform DirLight dirLight;

struct PointLight {    
    vec3 position;
    
    float constant;
    float linear;
    float quadratic;  

    vec3 ambient;
    vec3 diffuse;
    vec3 specular;
};  
#define NR_POINT_LIGHTS 1
uniform PointLight pointLights[NR_POINT_LIGHTS];


vec3 CalcDirLight(DirLight light, vec3 normal, vec3 viewDir)
{
    vec3 lightDir = normalize(-light.direction);
    // diffuse shading
    float diff = max(dot(normal, lightDir), 0.0);
    // specular shading
    vec3 reflectDir = reflect(-lightDir, normal);
    float spec = pow(max(dot(viewDir, reflectDir), 0.0), material.shininess);
    // combine results
    vec3 ambient  = light.ambient  * vec3(texture(material.diffuse, texcoord_0));
    vec3 diffuse  = light.diffuse  * diff * vec3(texture(material.diffuse, texcoord_0));
    //vec3 specular = light.specular * spec * vec3(texture(material.specular, texcoord_0));
    vec3 specular = light.specular * spec * vec3(0, 0, 0);
    return (ambient + diffuse + specular);
}


vec3 CalcPointLight(PointLight light, vec3 normal, vec3 fragPos, vec3 viewDir)
{
    vec3 lightDir = normalize(light.position - fragPos);
    // diffuse shading
    float diff = max(dot(normal, lightDir), 0.0);
    // specular shading
    vec3 reflectDir = reflect(-lightDir, normal);
    float spec = pow(max(dot(viewDir, reflectDir), 0.0), material.shininess);
    // attenuation
    float distance    = length(light.position - fragPos);
    float attenuation = 1.0 / (light.constant + light.linear * distance + 
                               light.quadratic * (distance * distance));    
    // combine results
    vec3 ambient  = light.ambient  * vec3(texture(material.diffuse, texcoord_0));
    vec3 diffuse  = light.diffuse  * diff * vec3(texture(material.diffuse, texcoord_0));
    //vec3 specular = light.specular * spec * vec3(texture(material.specular, texcoord_0));
    vec3 specular = light.specular * spec * vec3(0, 0, 0);
    ambient  *= attenuation;
    diffuse  *= attenuation;
    specular *= attenuation;
    return (ambient + diffuse + specular);
} 


void main()
{
    vec3 viewPos = vec3(3, 3, 3);

    // properties
    vec3 norm = normalize(normal);
    vec3 viewDir = normalize(viewPos - position);

    // phase 1: Directional lighting
    vec3 result = CalcDirLight(dirLight, norm, viewDir);
    
    // phase 2: Point lights
    //for(int i = 0; i < NR_POINT_LIGHTS; i++)
    //    result += CalcPointLight(pointLights[i], norm, position, viewDir);    
    
    // phase 3: Spot light
    //result += CalcSpotLight(spotLight, norm, position, viewDir);    
    
    result += vec3(1.0, 0.0, 0.0);
    
    FragColor = vec4(result, 1.0);
}
''',
callback=callback,
signature=('in_position', 'in_texcoord_0'))
#signature=('in_normal', 'in_texcoord_0', 'in_position'))

win = GLWindow.test('../resources/textures', shaders=shaders)

Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


In [ ]:

geometry_shader=''' 
#version 330 core
layout (triangles) in;
layout (triangle_strip, max_vertices = 3) out;

in VS_OUT {
    vec2 texCoords;
} gs_in[];

out vec2 TexCoords; 

uniform float time;

vec4 explode(vec4 position, vec3 normal) { ... }

vec3 GetNormal() { ... }

void main() {    
    vec3 normal = GetNormal();

    gl_Position = explode(gl_in[0].gl_Position, normal);
    TexCoords = gs_in[0].texCoords;
    EmitVertex();
    gl_Position = explode(gl_in[1].gl_Position, normal);
    TexCoords = gs_in[1].texCoords;
    EmitVertex();
    gl_Position = explode(gl_in[2].gl_Position, normal);
    TexCoords = gs_in[2].texCoords;
    EmitVertex();
    EndPrimitive();
}
''',

In [ ]:




struct LightInfo {
  vec4 Position;  // Light position in eye coords.
  vec3 Intensity; // A,D,S intensity
};
uniform LightInfo Light;

struct MaterialInfo {
  vec3 Ka;            // Ambient reflectivity
  vec3 Kd;            // Diffuse reflectivity
  vec3 Ks;            // Specular reflectivity
  float Shininess;    // Specular shininess factor
};
uniform MaterialInfo Material;

void phongModel( vec3 pos, vec3 norm, out vec3 ambAndDiff, out vec3 spec ) {

    vec3 s = vec3(0, 0, 0);
    if(Light.Position.w == 0.0) {
        s = normalize(-vec3(Light.Position));
    }
    else {
        s = normalize(vec3(Light.Position) - pos);
    }
    
    vec3 v = normalize(-pos.xyz);
    vec3 r = reflect( -s, norm );
    vec3 ambient = Light.Intensity * Material.Ka;
    float sDotN = max( dot(s,norm), 0.0 );
    vec3 diffuse = Light.Intensity * Material.Kd * sDotN;
    spec = vec3(0.0);
    if( sDotN > 0.0 )
        spec = Light.Intensity * Material.Ks *
               pow( max( dot(r,v), 0.0 ), Material.Shininess );
    ambAndDiff = ambient + diffuse;
    //ambAndDiff = vec3(100, 100, 100);
}


void main() {
    vec3 ambAndDiff, spec;
    vec4 texColor = texture( Tex1, TexCoord );
    phongModel( Position, Normal, ambAndDiff, spec );
    FragColor = (vec4( ambAndDiff, 1.0 ) * texColor) + vec4(spec,1.0);
}

In [3]:
dir(win.prog)

AttributeError: 'NoneType' object has no attribute 'prog'

In [9]:
win.prog._members

{'VertexPosition': <Attribute: 0>,
 'VertexNormal': <Attribute: 1>,
 'VertexTexCoord': <Attribute: 2>,
 'ModelViewMatrix': <Uniform: 0>,
 'NormalMatrix': <Uniform: 1>,
 'MVP': <Uniform: 2>,
 'Tex1': <Uniform: 3>,
 'Light.Position': <Uniform: 4>,
 'Light.Intensity': <Uniform: 5>,
 'Material.Ka': <Uniform: 6>,
 'Material.Kd': <Uniform: 7>,
 'Material.Ks': <Uniform: 8>,
 'Material.Shininess': <Uniform: 9>}